In [9]:
# test reading headers
import csv
with open('testdata/csv_test_in_001.csv', encoding = 'utf-8-sig', errors = 'ignore', newline = '') as file_in:
    reader = csv.reader(file_in)
    rowcount = 0
    rowsprocessed = 0
    headers = {}
    with open('csv_test_out_001.csv', mode = 'w') as file_out:
        writer = csv.writer(file_out, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
        for row in reader:
            if rowcount == 0:
                for cell in row:
                    header = {row.index(cell): cell}
                    headers.update(header)
                rowcount += 1
            else:
                pass
print(headers)

{0: 'id', 1: 'string', 2: 'attribute', 3: 'value', 4: 'comment'}


In [10]:
# test writing new headers
import csv
with open('testdata/csv_test_in_001.csv', encoding = 'utf-8-sig', errors = 'ignore', newline = '') as file_in:
    reader = csv.reader(file_in)
    rowcount = 0
    rowsprocessed = 0
    headers = {}
    with open('../testdata/csv_test_out_001.csv', mode = 'w') as file_out:
        writer = csv.writer(file_out, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
        newheaders = ['id', 'value', 'header']
        writer.writerow(newheaders)

In [1]:
import csv
with open('testdata/csv_test_in_001.csv', encoding = 'utf-8-sig', errors = 'ignore', newline = '') as file_in:
    reader = csv.reader(file_in)
    rowcount = 0
    rowsprocessed = 0
    headers = {}
    with open('../testdata/csv_test_out_001.csv', encoding = 'utf-8-sig', mode = 'w') as file_out:
        writer = csv.writer(file_out, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
        newheaders = ['id', 'value', 'header']
        writer.writerow(newheaders)
        for row in reader:
            if rowcount == 0:
                for cell in row:
                    header = {row.index(cell): cell}
                    headers.update(header)
                rowcount += 1
            elif rowcount > 0 and row[0] != '':
                for cell in row:
                    newrow = []
                    if row.index(cell) == 0: # accessing the position of the cell in the row
                        pass
                    else:
                        newrow.append(row[0])
                        newrow.append(cell)
                        newrow.append(headers[row.index(cell)])
                        writer.writerow(newrow)


# TO DO
- Eliminate every-other-row blank in output
    - Although, this doesn't show up in the [online rendering of the output CSV](https://gist.github.com/briesenberg07/1d3bdc9d079a8581768088dea1111f6b/cc7bca414690ff98ee0b24ffd09a38ffe2f55163#file-csv_test_out_001-csv), which is strange
    - What could be creating this (at least in the output file viewed in Excel)??

# SOLVED
- Garbled characters
    - On first run there were garbled characters, but changing the `open()` encoding argument for the `csv.writer` object to match that of the `open()` encoding arg for the `csv.reader` object ('utf-8-sig') fixed it!
- Don't process blank rows
    - See `elif rowcount > 0 and row[0] != '':`


# NOTE
- Error when attempting to process while `csv_test_out_001` is open in Excel:

```
PermissionError: [Errno 13] Permission denied: '../testdata/csv_test_out_001.csv'
```